# Clustering

## 1. K-means

### Préparation

On suit la même démarche que celle utilisée en TP pour tenter un premier clustering avec k-means.

In [ ]:
import pandas as pd

df = pd.read_parquet("data_cleaned.parquet")
df = df.dropna(subset=["lat", "long"]).copy()

df_kmeans = df[["lat", "long"]].copy()

df_kmeans.head()

On standardise les données géographiques (lat, long) afin que le clustering ne soit pas biaisé par l'échelle des coordonnées. Ainsi, la latitude et la longitude auront la même importance dans le calcul des distances.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_kmeans)

scaled_data_df = pd.DataFrame(scaled_data, columns=df_kmeans.columns, index=df_kmeans.index)
scaled_data_df.head()

### Fonctions de base

On définit nos deux fonctions permettant d'exécuter le clustering et de le visualiser.

La fonction run_kmeans_clustering exécute le clustering k-means pour une valeur donnée de k.

In [ ]:
from sklearn.cluster import KMeans

def run_kmeans_clustering(df, scaled_data_df, scaler, k):
    kmeans = KMeans(n_clusters=k, init="k-means++", random_state=0)
    kmeans.fit(scaled_data_df)
    
    df["cluster_kmeans"] = kmeans.labels_
    
    # Compute centers in original space
    centers_scaled = kmeans.cluster_centers_
    centers = scaler.inverse_transform(centers_scaled)
    centers_df = pd.DataFrame(centers, columns=["lat", "long"])
    
    return centers_df

On définit la fonction create_clustering_map qui crée une carte permettant de visualiser les clusters obtenus.

In [ ]:
import folium
from scipy.spatial import ConvexHull
import numpy as np

def create_clustering_map(df, cluster_col, centers_df=None, sample_size=5000, zoom_start=12):    
    sample = df.sample(n=min(sample_size, len(df)), random_state=0)
    
    palette = [
        "red", "blue", "green", "purple", "orange",
        "darkred", "lightred", "beige", "darkblue",
        "darkgreen", "cadetblue", "darkpurple",
        "pink", "lightblue", "lightgreen",
        "gray", "black", "lightgray"
    ]
    
    m = folium.Map(
        location=[df["lat"].median(), df["long"].median()],
        zoom_start=zoom_start,
        tiles="CartoDB positron",
    )
    
    # Dessiner les contours des clusters
    for cluster_id in df[cluster_col].unique():
        if cluster_id == -1:  # Ignorer le bruit
            continue
        cluster_points = df[df[cluster_col] == cluster_id][["lat", "long"]].values
        if len(cluster_points) > 2:
            try:
                hull = ConvexHull(cluster_points)
                hull_coords = cluster_points[hull.vertices]
                color = palette[cluster_id % len(palette)]
                folium.PolyLine(
                    locations=[[lat, lon] for lat, lon in hull_coords],
                    color=color,
                    weight=2,
                    opacity=0.8
                ).add_to(m)
            except:
                pass
    
    # Points d'échantillon (moins nombreux)
    for _, r in sample.iterrows():
        c = int(r[cluster_col])
        color = palette[c % len(palette)]
        folium.CircleMarker(
            location=[r["lat"], r["long"]],
            radius=2,
            color=color,
            fill=True,
            fill_opacity=0.5,
            popup=folium.Popup(f"""<a href="{r.get('url','')}" target="_blank">Open Flickr</a>""", max_width=250),
        ).add_to(m)
    
    # Centres des clusters
    if centers_df is not None:
        for i, row in centers_df.iterrows():
            folium.Marker(
                location=[row["lat"], row["long"]],
                icon=folium.Icon(color="darkblue", icon="star"),
                popup=f"Centre du cluster {i}",
            ).add_to(m)
    
    return m

### K déterminé par la méthode du coude

On commence par déterminer le nombre optimal de clusters k en utilisant la méthode du coude. On calcule l'inertie pour différentes valeurs de k et on trace la courbe correspondante.

L'inertie mesure la dispersion des points au sein des clusters : plus les points sont proches de leur centre de cluster, plus l'inertie est faible.

In [ ]:
import matplotlib.pyplot as plt

inertia_values = []
k_values = range(1, 15)

for k in k_values:
    kmeans = KMeans(n_clusters=k, init="k-means++", random_state=0)
    kmeans.fit(scaled_data_df)
    inertia_values.append(kmeans.inertia_)

plt.figure()
plt.plot(list(k_values), inertia_values, marker="o")
plt.title("Méthode du coude pour K-Means")
plt.xlabel("k")
plt.ylabel("Inertie")
plt.show()

On trouve k = 4 avec la méthode du coude. On teste un clustering avec cette valeur.

In [ ]:
centers_df = run_kmeans_clustering(df, scaled_data_df, scaler, 4)

In [ ]:
m = create_clustering_map(df, "cluster_kmeans", centers_df=centers_df)
m.save("output/kmeans/kmeans4.html")

Le résultat obtenu avec k = 4 n'est pas satisfaisant. La méthode du coude nous donne une valeur de k qui ne produit de clustering pertinent.

Dans Lyon, il n'y a pas que 4 zones d'intérêt. Il faudrait essayer avec un k plus grand.

### Autres valeurs de K

In [ ]:
for k in [10, 50, 150, 200]:
    centers_df = run_kmeans_clustering(df, scaled_data_df, scaler, k)
    m = create_clustering_map(df, "cluster_kmeans", centers_df=centers_df)
    m.save(f"output/kmeans/kmeans{k}.html")

Même avec d'autres valeurs de k, le clustering n'est pas vraiment pertinent.

### Analyse des résultats

Même avec d'autres valeurs de k, le clustering K-means n'est pas adapté à notre cas d'usage.

**Limites observées** :
- K-means **partitionne l'espace géographique** de manière uniforme plutôt que d'identifier les zones de forte densité
- Les clusters sont **forcés d'être sphériques** (distances euclidiennes), inadapté aux hotspots touristiques de formes irrégulières
- **Sensibilité au bruit** : les points isolés influencent les centres des clusters
- **Nécessité de fixer k a priori** sans garantie sur la pertinence du nombre de zones d'intérêt

K-means privilégie une **couverture territoriale homogène** plutôt que la détection de **hotspots denses**, ce qui ne répond pas à notre objectif d'optimisation des transports.

## 2.2 DBSCAN

Suite à l'essai de k-means, on tente un clustering avec DBSCAN.

Le principe de DBSCAN est de regrouper les points denses ensemble, et de considérer les points isolés comme du bruit.

### Préparation

On importe les données nettoyées.

In [ ]:
import pandas as pd

df = pd.read_parquet("output/data_cleaned.parquet")
df = df.dropna(subset=["lat", "long"]).copy()

In [ ]:
import numpy as np

coords = df[["lat", "long"]].to_numpy()
coords_rad = np.radians(coords)

### Fonctions de base

On crée une fonction DBSCAN pour pouvoir l'appliquer avec différents paramètres.

Afin de pouvoir utiliser un epsilon exprimé en mètres, on convertit les coordonnées géographiques (lat, long) en coordonnées cartésiennes, en utilisant haversine.

In [ ]:
from sklearn.cluster import DBSCAN

def run_dbscan(coords_rad, eps_meters, min_samples):
    # conversion mètres → radians
    eps_rad = eps_meters / 6371000

    db = DBSCAN(
        eps=eps_rad,
        min_samples=min_samples,
        metric="haversine",
        algorithm="ball_tree" # Plus rapide pour haversine
    )
    labels = db.fit_predict(coords_rad)
    return labels

On crée une fonction qui nous permettra de visualiser les résultats du clustering DBSCAN sur une carte.

In [ ]:
import folium

def plot_dbscan_map(df, labels, title, sample_size=5000):
    dff = df.copy()
    dff["cluster"] = labels

    # échantillon
    if len(dff) > sample_size:
        dff = dff.sample(sample_size, random_state=0)

    m = folium.Map(
        location=[dff["lat"].median(), dff["long"].median()],
        zoom_start=12,
        tiles="CartoDB positron"
    )

    palette = [
        "red","blue","green","purple","orange","darkred","cadetblue",
        "darkgreen","darkpurple","pink","gray","black"
    ]

    for _, r in dff.iterrows():
        if r["cluster"] == -1:
            color = "lightgray"
        else:
            color = palette[r["cluster"] % len(palette)]

        folium.CircleMarker(
            location=[r["lat"], r["long"]],
            radius=2,
            color=color,
            fill=True,
            fill_opacity=0.6,
        ).add_to(m)

    return m

In [ ]:
import numpy as np
from sklearn.metrics import silhouette_score

def dbscan_stats(coords_rad, labels):
    n_noise = np.sum(labels == -1)
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

    # Silhouette uniquement sur points non-bruit
    mask = labels != -1
    sil = None
    if mask.sum() > 1 and n_clusters > 1:
        sil = silhouette_score(coords_rad[mask], labels[mask], metric="euclidean")

    return {
        "clusters": n_clusters,
        "noise_ratio": n_noise / len(labels),
        "silhouette": sil,
    }

### Détermination des paramètres epsilon et min_samples

#### Min_samples

La littérature indique que min_samples peut être fixé à 4 pour des données en 2D. On commence donc notre analyse avec cette valeur.

#### Epsilon déterminé avec la méthode du coude

On veut déterminer une bonne valeur pour epsilon en utilisant la méthode du coude.
Pour cela, on calcule la distance au 4ème plus proche voisin pour chaque point, puis on trace ces distances triées.

4 correspond au min_samples que l'on souhaite utiliser pour DBSCAN.

In [ ]:
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

def coude_dbscan(min_samples, coords_rad):
    k = min_samples  # min_samples value
    neighbors = NearestNeighbors(n_neighbors=k)
    neighbors_fit = neighbors.fit(coords_rad)
    distances, indices = neighbors_fit.kneighbors(coords_rad)

    distances = np.sort(distances[:, k-1], axis=0)

    # Tracer la courbe
    plt.figure(figsize=(10, 6))
    plt.plot(distances)
    plt.ylabel(f'Distance au {k}-ième plus proche voisin (radians)')
    plt.xlabel('Points (triés par distance)')
    plt.title('Méthode du coude pour déterminer epsilon (DBSCAN)')
    plt.grid(True)

    return plt

coude_plot = coude_dbscan(min_samples=4, coords_rad=coords_rad)
coude_plot.axhline(y=90/6371000, color='r', linestyle='--', label='90m')
coude_plot.legend()
coude_plot.show()

On choisit donc de commencer notre analyse avec epsilon = 90m et min_samples = 4.

### Avec epsilon = 90m et min_samples = 4 

In [ ]:
results = []

labels = run_dbscan(coords_rad, 90, 4)
plot_dbscan_map(df, labels, title="DBSCAN eps=90m / min_samples=4").save("output/dbscan/dbscan_eps90_ms4.html")

On observe :
- Beaucoup de clusters peu pertinents (trop peu de points pour être considéré comme intéressants).
- Des clusters beaucoup trop gros.

On va donc essayer DBSCAN avec :
- Des valeurs d'epsilon plus petites.
- Des valeurs de min_samples plus grandes.

### En variant epsilon et min_samples

In [ ]:
results = []

for eps in [20, 40, 60]:
    for ms in [5, 50, 100]:
        labels = run_dbscan(coords_rad, eps, ms)
        map = plot_dbscan_map(df, labels, title=f"DBSCAN eps={eps}m / min_samples={ms}")
        map.save(f"output/dbscan/dbscan_eps{eps}_ms{ms}.html")

### Analyse des résultats

Les tests avec différentes combinaisons de paramètres révèlent **une limitation fondamentale de DBSCAN** pour notre cas d'usage.

**Observations :**

1. **Impact de min_samples** : Augmenter `min_samples` améliore significativement la pertinence des clusters en filtrant les zones peu denses. Un seuil élevé (50-100) permet d'identifier uniquement les véritables hotspots touristiques.

2. **Dilemme du paramètre epsilon** : Le choix d'epsilon pose un problème structurel pour Lyon :
   - **Epsilon élevé (~100m)** : Fusionne les points d'intérêt du centre-ville (ex: fusion de la Presqu'île en un seul mega-cluster), perdant la granularité nécessaire pour distinguer Place Bellecour, Place des Terreaux, etc.
   - **Epsilon faible (~30m)** : Capture bien les petits clusters denses du centre, mais échoue à détecter les zones d'intérêt plus dispersées (Parc de la Tête d'Or, quartier de la Part-Dieu).

**Conclusion** : DBSCAN nécessite un epsilon **uniforme** alors que Lyon présente des zones d'intérêt de **densités variables**. Cette rigidité empêche de capturer simultanément les petits clusters concentrés du centre-ville et les grands espaces touristiques périphériques.

→ **Solution** : On utilise donc HDBSCAN qui adapte automatiquement la distance selon la densité locale.

## 2.3 HDBSCAN

HDBSCAN adapte la densité locale, il n'est donc pas nécessaire de fixer un epsilon global. On espère ainsi mieux capturer les zones d'intérêt de densités variées à Lyon.

**Paramètre clé** : `min_cluster_size` détermine la taille minimale d'un cluster.

### Préparation
Pour HDBSCAN, on travaille en coordonnées cartésiennes (projection Web Mercator EPSG:3857) plutôt qu'en lat/long, pour avoir des distances en mètres. GeoPandas nous permet de faire cette conversion facilement.

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd

# Charger les données nettoyées
df = pd.read_parquet("output/data_cleaned.parquet")
df = df.dropna(subset=["lat", "long"]).copy()

# Convertir en GeoDataFrame avec projection Web Mercator
gdf = gpd.GeoDataFrame(
    df, 
    geometry=gpd.points_from_xy(df["long"], df["lat"]), 
    crs="EPSG:4326"
)
gdf = gdf.to_crs("EPSG:3857")

# Extraire les coordonnées cartésiennes en mètres
X_m = np.column_stack([gdf.geometry.x.to_numpy(), gdf.geometry.y.to_numpy()])

print(f"Données préparées : {len(X_m)} points avec coordonnées cartésiennes")

### Fonctions de base
On crée trois fonctions réutilisables :
1. **run_hdbscan** : Exécute le clustering HDBSCAN
2. **create_hdbscan_map** : Visualise les résultats sur une carte
3. **hdbscan_stats** : Calcule les statistiques des clusters

In [ ]:
import hdbscan

def run_hdbscan(X_m, min_cluster_size, min_samples=None):
    """
    Exécute HDBSCAN sur les coordonnées cartésiennes.
    
    Args:
        X_m : Coordonnées en mètres (n_samples, 2)
        min_cluster_size : Taille minimale d'un cluster
        min_samples : Nombre minimal de points pour un core point (None = min_cluster_size)
    
    Returns:
        labels : Étiquettes de cluster (-1 pour bruit)
    """
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=min_cluster_size,
        min_samples=min_samples,
    )
    labels = clusterer.fit_predict(X_m)
    return labels

In [ ]:
import folium

def create_hdbscan_map(df, labels, title="HDBSCAN Clustering", sample_size=1000, zoom_start=12):
    """
    Crée une carte Folium pour visualiser les clusters HDBSCAN.
    
    Args:
        df : DataFrame avec colonnes 'lat' et 'long'
        labels : Tableau d'étiquettes de cluster
        title : Titre de la visualisation
        sample_size : Nombre de points à afficher (pour performance)
        zoom_start : Niveau de zoom initial
    
    Returns:
        m : Objet folium.Map
    """
    dff = df.copy()
    dff["cluster"] = labels
    
    # Échantillonner si trop de points
    if len(dff) > sample_size:
        dff = dff.sample(n=sample_size, random_state=0)
    
    m = folium.Map(
        location=[dff["lat"].median(), dff["long"].median()],
        zoom_start=zoom_start,
        tiles="CartoDB positron"
    )
    
    palette = [
        "red", "blue", "green", "purple", "orange",
        "darkred", "lightred", "beige", "darkblue",
        "darkgreen", "cadetblue", "darkpurple",
        "pink", "lightblue", "lightgreen",
        "gray", "black", "lightgray"
    ]
    
    for _, r in dff.iterrows():
        if r["cluster"] == -1:
            # Ignorer les points de bruit
            continue
        else:
            color = palette[r["cluster"] % len(palette)]
        
        folium.CircleMarker(
            location=[r["lat"], r["long"]],
            radius=2,
            color=color,
            fill=True,
            fill_opacity=0.6,
            popup=folium.Popup(
                f"""<a href="{r.get('url', '')}" target="_blank">Open Flickr</a>""",
                max_width=250
            )
        ).add_to(m)
    
    return m

In [ ]:
def hdbscan_stats(labels):
    """
    Calcule les statistiques du clustering HDBSCAN.
    
    Args:
        labels : Tableau d'étiquettes de cluster
    
    Returns:
        dict avec les statistiques
    """
    n_noise = np.sum(labels == -1)
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    noise_ratio = n_noise / len(labels) * 100
    
    return {
        "n_clusters": n_clusters,
        "n_noise": n_noise,
        "noise_ratio": noise_ratio,
        "total_points": len(labels),
    }

### Détermination des paramètres
Le paramètre clé est `min_cluster_size`. Une valeur trop faible crée du bruit, une valeur trop élevée écrase les petits clusters pertinents.

On teste plusieurs valeurs pour trouver l'équilibre optimal : 20, 50, 100, 200.

In [ ]:
# Tester différentes valeurs de min_cluster_size
results_hdbscan = []

for min_cluster_size in [20, 50, 100, 200]:
    labels = run_hdbscan(X_m, min_cluster_size=min_cluster_size)
    stats = hdbscan_stats(labels)
    results_hdbscan.append(stats)
    
    print(f"\nmin_cluster_size = {min_cluster_size}")
    print(f"  Clusters : {stats['n_clusters']}")
    print(f"  Points bruit : {stats['n_noise']} ({stats['noise_ratio']:.1f}%)")
    
    # Générer et sauvegarder la carte
    m = create_hdbscan_map(df, labels, title=f"HDBSCAN min_cluster_size={min_cluster_size}")
    m.save(f"output/hdbscan/hdbscan_mc{min_cluster_size}.html")

### Résultats
Basé sur les statistiques et les visualisations, on choisit `min_cluster_size=100`, qui semble être le paramètre avec lequel HDBSCAN produit les meilleurs clusters.

### Analyse des résultats et comparaison

**Comparaison des trois méthodes** :

| Critère | K-means | DBSCAN | HDBSCAN |
|---------|---------|--------|---------|
| Adaptation densité variable | ❌ Non | ⚠️ Partiel | ✅ Oui |
| Formes clusters | Sphériques | Quelconques | Quelconques |
| Paramétrage | k requis | epsilon+ms | min_cluster_size |
| Sensibilité bruit | Haute | Moyenne | Basse |
| **Pertinence Lyon** | Faible | Moyen | **Excellente** |

**Conclusion** : HDBSCAN capture efficacement les hotspots de Lyon en s'adaptant à leurs densités variables, tout en gérant bien le bruit. C'est donc notre meilleure option pour l'optimisation des transports touristiques.

### Export des données

In [ ]:
# Ajouter les labels finaux au DataFrame
optimal_min_cluster_size = 100
labels_final = run_hdbscan(X_m, min_cluster_size=optimal_min_cluster_size)
df["cluster_hdbscan"] = labels_final

# Exporter en Parquet pour utilisation ultérieure
df.to_parquet("output/flickr_data_clustered.parquet")

print(f"✓ Données exportées : {len(df)} points avec cluster_hdbscan")
print(f"✓ Clusters identifiés : {final_stats['n_clusters']}")